# **RVC AI Cover Maker (separate parts version)**
Created by [ShiromiyaG](https://github.com/ShiromiyaG)
- Colab inspired on [AICoverGen](https://github.com/SociallyIneptWeeb/AICoverGen) by [SociallyIneptWeeb](https://github.com/SociallyIneptWeeb)
- Uses the [blaise-tk](https://github.com/blaise-tk) version of [RVC_CLI](https://github.com/blaise-tk/RVC_CLI)

In [ ]:
import tarfile, zipfile
import os
import sys
from google.colab import drive
from pathlib import Path
import contextlib

def extract_tar_file(tar_file_path, destination_path):
  if not Path(tar_file_path).exists():
      print(f"Tar file {tar_file_path} does not exist.")
      return

  extraction_failed = False

  with tarfile.open(tar_file_path, "r:gz") as tar:
      for member in tar.getmembers():
          try:
              tar.extract(member, destination_path)
          except Exception as e:
              print(f"Failed to extract file {member.name}: {e}")
              extraction_failed = True

  print(f"Extraction of {tar_file_path} to {destination_path} completed.")

  if os.path.exists(tar_file_path):
      try:
          os.remove(tar_file_path)
      except Exception as e:
          print(f"Failed to remove tar file {tar_file_path}: {e}")

  if extraction_failed:
      print("Extraction encountered errors.")

@contextlib.contextmanager
def supress_output(supress=True):
    if supress:
        with open(os.devnull, "w") as devnull:
            old_stdout = sys.stdout
            old_stderr = sys.stderr
            sys.stdout = devnull
            sys.stderr = devnull
            try:
                yield
            finally:
                sys.stdout = old_stdout
                sys.stderr = old_stderr
    else:
        yield

drive.mount('/content/drive')
Path('/content/dependencies').mkdir(parents=True, exist_ok=True)
os.chdir("/content")
# @title # **Install**
# @markdown ##### Yes, it takes time
# @markdown ##### To find bugs, you have the option of not clearing the output
not_clean_outputs = False #@param {type:"boolean"}
# @markdown ##### Install precompiled pip dependencies for faster installation **(May be dated)**
precompiled = True #@param {type:"boolean"}
supress = True
with supress_output(not not_clean_outputs):
  !sudo apt-get install ffmpeg python3.10-venv aria2 libsamplerate0-dev -y
  !git clone https://github.com/jarredou/MVSEP-MDX23-Colab_v2.git
  if not precompiled:
    !pip install MVSEP-MDX23-Colab_v2/requirements.txt

def install_project(supress):
  with supress_output(supress):
    !wget -P /content "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/inference.py"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/mdx23colab-0.1.0-py3-none-any.whl"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/rvccli-0.1.0-py3-none-any.whl"
    !wget -P /content "https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/shiromiyautils-0.1.0-py3-none-any.whl"
    !pip install mdx23colab-0.1.0-py3-none-any.whl
    !pip install rvccli-0.1.0-py3-none-any.whl
    !pip install shiromiyautils-0.1.0-py3-none-any.whl

def install_precompiled_dependencies(supress=True):
  with supress_output(supress):
    arquivo_zip = '/content/RVCAICoverMakerDeps.tar.gz'
    pasta_destino = '/content/dependencies'
    Path('/content/arquivos').mkdir(parents=True, exist_ok=True)
    !aria2c --console-log-level=error -x 16 -s 16 -k 1M https://huggingface.co/ShiromiyaGamer/dependencias/resolve/main/RVCAICoverMakerDeps.tar.gz -d /content -o RVCAICoverMakerDeps.tar.gz
    source_path = "/content/RVCAICoverMakerDeps.tar.gz"
    destination_path = "/"
    extract_tar_file(source_path, destination_path)

def install_dependencies(precompiled=True, supress=True):
  with supress_output(supress):
    if not precompiled:
      !pip install poetry pedalboard
      !poetry config virtualenvs.create false
      !pip install --no-deps pyrubberband dora-search retrying hydra-core>=1.1
      !pip install yt-dlp[default] wget git+https://github.com/IAHispano/gdown
      !pip install --upgrade --upgrade-strategy only-if-needed pydub soxr cmake audiofile samplerate==0.1.0
      !pip install python-dotenv samplerate==0.1.0

def setup_ultimatevocalremover_api(supress=True):
  with supress_output(supress):
    !git clone https://github.com/NextAudioGen/ultimatevocalremover_api.git
    !rm ultimatevocalremover_api/requirements.txt
    !rm ultimatevocalremover_api/src/models_dir/models.json
    !rm ultimatevocalremover_api/src/models_dir/mdx/modelparams/model_data.json
    !rm ultimatevocalremover_api/src/models_dir/vr_network/modelparams/model_data.json
    !wget -P ultimatevocalremover_api/src/models_dir/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/models.json"
    !wget -P ultimatevocalremover_api/src/models_dir/mdx/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/mdx/model_data.json"
    !wget -P ultimatevocalremover_api/src/models_dir/vr_network/modelparams/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/vr/model_data.json"
    !wget -P ultimatevocalremover_api/ "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/UVRapi/requirements.txt"
    %cd ultimatevocalremover_api
    !pip install --upgrade --upgrade-strategy only-if-needed .
    %cd /content

def setup_kitware(supress=True):
  with supress_output(supress):
    !wget -O - https://apt.kitware.com/keys/kitware-archive-latest.asc 2>/dev/null | gpg --dearmor - | sudo tee /etc/apt/trusted.gpg.d/kitware.gpg >/dev/null
    !sudo apt-add-repository "deb https://apt.kitware.com/ubuntu/ $(lsb_release -cs) main" -y
    !sudo apt update
    !sudo apt install kitware-archive-keyring
    !sudo rm /etc/apt/trusted.gpg.d/kitware.gpg
    !sudo apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 6AF7F09730B3F0A4
    !sudo apt update
    !sudo apt-get install libsamplerate0-dev g++ make cmake

def setup_orpheusdl(precompiled=True, supress=True):
  with supress_output(supress):
    !git clone https://github.com/OrfiTeam/OrpheusDL.git
    if not precompiled:
      %cd OrpheusDL
      !poetry install --no-root -q > /dev/null 2>&1
      %cd /content
    !git clone https://git.ovosimpatico.com/ovosimpatico/orpheusdl-deezer.git ./OrpheusDL/modules/deezer
    !wget -P OrpheusDL/config "https://raw.githubusercontent.com/ShiromiyaG/RVC-AI-Cover-Maker/v2/OrpheusDL/settings.json"

def setup_rvc_cli(precompiled=True, supress=True):
  with supress_output(supress):
    os.chdir("/content")
    !wget -P /content "https://github.com/blaise-tk/RVC_CLI/archive/refs/tags/1.1.1.zip"
    extraction_folder = os.path.join("/content")
    with zipfile.ZipFile("/content/1.1.1.zip", 'r') as zip_ref:
        zip_ref.extractall(extraction_folder)
    os.rename("/content/RVC_CLI-1.1.1", "/content/RVC_CLI")
    %cd RVC_CLI
    if not precompiled:
      !poetry install --no-root > /dev/null 2>&1
    !python main.py prerequisites
    %cd /content

def main(precompiled=True,not_clean_outputs=False):
  if not not_clean_outputs:
    supress = True
  else:
    supress = False
  print("Instalando dependências...")
  with supress_output(supress):
    install_project(supress)
    install_dependencies(precompiled,supress)
    if precompiled:
      !pip uninstall numpy==1.26.4 -y
      !pip uninstall numpy==1.25.2 -y
      install_precompiled_dependencies(supress)
    else:
      setup_ultimatevocalremover_api(supress)
    setup_kitware(supress)
    setup_orpheusdl(precompiled, supress)
    setup_rvc_cli(precompiled, supress)
    !pip uninstall onnxruntime-gpu -y
    !python -m pip -q install onnxruntime-gpu --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
    !pip install --upgrade --force-reinstall numpy==1.23.5
  print("Installed dependencies")

main(precompiled, not_clean_outputs)

# **Send audio file**

In [ ]:
#@title ### **Local file upload**
from google.colab import files
import os
import fnmatch
from pathlib import Path
Path('/content/drive/MyDrive/RVCAICoverMakerInput').mkdir(parents=True, exist_ok=True)
def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".flac") or file.endswith(".wav"):
                caminho_arquivo = os.path.join(root, file)
                arquivos.append(caminho_arquivo)
    return arquivos
uploaded = files.upload()
uploaded = procurar_arquivos("/content")
for arquivo in uploaded:
  !cp "{arquivo}" "/content/drive/MyDrive/RVCAICoverMakerInput"

In [ ]:
#@title ### **Files from Google Drive**
import os
import fnmatch
from google.colab import drive
from pathlib import Path
import importlib
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
drive_path = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}

def procurar_arquivos(pasta):
    arquivos = []
    for root, dirs, files in os.walk(pasta):
        for file in files:
            if fnmatch.fnmatch(file, '*.mp3') or fnmatch.fnmatch(file, '*.flac') or fnmatch.fnmatch(file, '*.wav'):
                arquivos.append(os.path.join(root, file))
    return arquivos

def pacote_instalado(nome_pacote):
    try:
        importlib.import_module(nome_pacote)
        return True
    except ImportError:
        return False

if pacote_instalado("audio-separator"):
  pass
else:
  print("Dependencies not installed, setting up the Drive now")
  drive.mount('/content/drive')

arquivos_encontrados = procurar_arquivos(drive_path)

for arquivo in arquivos_encontrados:
    !cp "{arquivo}" "/content/musicas/arquivos-originais"

In [ ]:
#@title ### **Download file from YouTube or Deezer**
from pathlib import Path
import os
# @markdown #### Select one of the options to download a song
link_of_yt = "" #@param {type:"string"}
link_of_deezer = "" #@param {type:"string"}
# @markdown ###### If you are going to use Deezer to download
bf_secret = "" #@param {type:"string"}
track_url_key = "" #@param {type:"string"}
arl = "" #@param {type:"string"}
Path('/content/musicas/arquivos-originais').mkdir(parents=True, exist_ok=True)
input_folder = '/content/musicas/arquivos-originais'

if link_of_yt != "":
  !python inference.py download_yt --link "{link_of_yt}" --supress True

if link_of_deezer != "":
  if bf_secret != "" and track_url_key != "" and arl != "":
    !python inference.py download_deezer --link "{link_of_deezer}" \
          --bf_secret "{bf_secret}" \
          --track_url_key "{track_url_key}" \
          --arl "{arl}" \
          --supress True
  else:
    raise ValueError("You need to provide the bf secret, track url key arl of your Deezer Premium account")

# **Programs**

In [ ]:
#@title ### **Remove Backing Vocals and Echo/Reverb**
from pathlib import Path
import os
from glob import glob
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" # @param {type:"string"}

no_back_folder = "/content/musicas/sem-back"
output_vocals_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(output_vocals_folder).mkdir(parents=True, exist_ok=True)
device = "cuda"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py remove_backing_vocals_and_reverb \
            --input_file "{input_file}" \
            --no_back_folder "{no_back_folder}" \
            --output_folder "{output_vocals_folder}" \
            --device "{device}" \
             --supress True


In [ ]:
#@title ### **Remove Instrumentals, Backing Vocals and Echo/Reverb**
# @markdown ##### Leave the imput folder like this to use the songs downloaded from YT/Deezer
from pathlib import Path
import os
from glob import glob
device = "cuda"
input_folder = "/content/musicas/arquivos-originais" # @param {type:"string"}
no_back_folder = "/content/musicas/sem-back"
no_inst_folder = "/content/musicas/sem-intrumental"
output_vocals_folder = "/content/musicas/output-vocals-folder"
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(no_back_folder).mkdir(parents=True, exist_ok=True)
Path(no_inst_folder).mkdir(parents=True, exist_ok=True)
Path(output_vocals_folder).mkdir(parents=True, exist_ok=True)
Path(output_drive).mkdir(parents=True, exist_ok=True)
# @markdown ---
# @markdown ### **Vocal Separation Options**
output_format = 'FLAC'
Separation_mode = 'Vocals/Instrumental'
input_gain = 0 #@param [0, -3, -6] {type:"raw"}
restore_gain_after_separation = False #@param {type:"boolean"}
filter_vocals_below_50hz = False #@param {type:"boolean"}
#@markdown  *Set BigShifts=1 to disable that feature*
BigShifts = 3 #@param {type:"slider", min:1, max:41, step:1}
BSRoformer_model = 'ep_317_1297' #@param ["ep_317_1297", "ep_368_1296"]
weight_BSRoformer = 10 #@param {type:"slider", min:0, max:10, step:1}
weight_InstVoc = 4 #@param {type:"slider", min:0, max:10, step:1}
use_VitLarge = True #@param {type:"boolean"}
weight_VitLarge = 1 #@param {type:"slider", min:0, max:10, step:1}
use_InstHQ4 = False #@param {type:"boolean"}
weight_InstHQ4 = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_InstHQ4 = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
use_VOCFT = False #@param {type:"boolean"}
weight_VOCFT = 2 #@param {type:"slider", min:0, max:10, step:1}
overlap_VOCFT = 0.1 #@param {type:"slider", min:0, max:0.95, step:0.05}
overlap_demucs = 0.6

use_InstVoc_ = '--use_InstVoc' #forced use
use_BSRoformer_ =  '--use_BSRoformer' #forced use
use_VOCFT_ = '--use_VOCFT' if use_VOCFT is True else ''
use_VitLarge_ = '--use_VitLarge' if use_VitLarge is True else ''
use_InstHQ4_ = '--use_InstHQ4' if use_InstHQ4 is True else ''
restore_gain = '--restore_gain' if restore_gain_after_separation is True else ''
vocals_only = '--vocals_only' if Separation_mode == 'Vocals/Instrumental' else ''
filter_vocals = '--filter_vocals' if filter_vocals_below_50hz is True else ''
input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Procesando", len(input_files), "arquivo" if len(input_files) == 1 else "arquivos")
  basename = os.path.basename(input_file).split(".")[0]
  %cd /content/MVSEP-MDX23-Colab_v2
  !python inference.py \
        --input_audio "{input_file}" \
        --large_gpu \
        --BSRoformer_model {BSRoformer_model} \
        --weight_BSRoformer {weight_BSRoformer} \
        --weight_InstVoc {weight_InstVoc} \
        --weight_InstHQ4 {weight_InstHQ4} \
        --weight_VOCFT {weight_VOCFT} \
        --weight_VitLarge {weight_VitLarge} \
        --overlap_demucs {overlap_demucs} \
        --overlap_VOCFT {overlap_VOCFT} \
        --overlap_InstHQ4 {overlap_InstHQ4} \
        --output_format {output_format} \
        --BigShifts {BigShifts} \
        --output_folder "{no_inst_folder}" \
        --input_gain {input_gain} \
        {filter_vocals} \
        {restore_gain} \
        {vocals_only} \
        {use_VitLarge_} \
        {use_VOCFT_} \
        {use_InstHQ4_} \
        {use_InstVoc_} \
        {use_BSRoformer_} 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_voc.txt
  output_vocals = get_last_modified_file("/content/musicas/sem-intrumental", vocals)
  %cd ..
  !python inference.py remove_backing_vocals_and_reverb \
          --input_file "{output_vocals}" \
          --no_back_folder "{no_back_folder}" \
          --output_folder "{output_drive}" \
          --device "{device}" \
          --supress True \
          --language "BR" 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_reverb.txt

In [ ]:
#@title ### **Remove Vocals (Keeping Backing Vocals)**
#@markdown ##### Leave the imput folder like this to use the songs downloaded from YT/Deezer
from pathlib import Path
import os
from glob import glob
input_folder = "/content/musicas/arquivos-originais" # @param {type:"string"}
stage1_dir = "/content/musicas/inst-etapa1"
stage2_dir = "/content/musicas/inst-etapa2"
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(stage1_dir).mkdir(parents=True, exist_ok=True)
Path(stage2_dir).mkdir(parents=True, exist_ok=True)
Path(output_drive).mkdir(parents=True, exist_ok=True)

instrumental_ensemble = True
device = "cuda"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py separate_instrumentals \
            --input_file "{input_file}" \
            --instrumental_ensemble {instrumental_ensemble} \
            --algorithm_ensemble_inst "Max Spec" \
            --stage1_dir "{stage1_dir}" \
            --stage2_dir "{stage2_dir}" \
            --final_output_dir "{output_drive}" \
            --device "{device}" \
            --supress True

In [ ]:
#@title ### **Vocals Effects**
from pathlib import Path
import os
from glob import glob
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
# @markdown ---
# @markdown ### **Reverb**
reverb = True # @param{type:"boolean"}
reverb_size = 0.15  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_wetness = 0.2 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_dryness = 0.8 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_damping = 0.7 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
reverb_width = 1.0 # @param {type:"slider", min:0.0, max:1.0, step:0.1}
# @markdown ### **Limiter**
limiter = False # @param{type:"boolean"}
limiter_threshold_db = 3.0 # @param {type:"slider", min:1.0, max:10.0, step:0.1}
limiter_release_ms = 60.0 # @param {type:"slider", min:1.0, max:5000.0, step:0.1}
# @markdown ### **Compressor**
compressor = False # @param{type:"boolean"}
compressor_ratio = 1.0 # @param {type:"slider", min:1.0, max:20.0, step:0.0}
compressor_threshold_db = -45.0 # @param {type:"slider", min:-60.0, max:0.0, step:0.1}
compressor_attack_ms = 2.0 # @param {type:"slider", min:0.1, max:200.0, step:0.1}
compressor_release_ms = 50.0 # @param {type:"slider", min:1.0, max:5000.0, step:0.1}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(output_folder).mkdir(parents=True, exist_ok=True)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py reverb \
          --audio_path "{input_folder}" \
          --reverb {reverb} \
          --reverb_size {reverb_size} \
          --reverb_wetness {reverb_wetness} \
          --reverb_dryness {reverb_dryness} \
          --reverb_damping {reverb_damping} \
          --reverb_width {reverb_width} \
          --limiter {limiter} \
          --limiter_threshold_db {limiter_threshold_db} \
          --limiter_release_ms {limiter_release_ms} \
          --compressor {compressor} \
          --compressor_ratio {compressor_ratio} \
          --compressor_threshold_db {compressor_threshold_db} \
          --compressor_attack_ms {compressor_attack_ms} \
          --compressor_release_ms {compressor_release_ms} \
          --output_path "{output_reverb_folder}/{basename}_output_reverb.wav" \
          --supress {supress} 2>> /content/drive/MyDrive/RVCAICoverMakerOutput/RVClogs/log_reverb.txt

In [ ]:
#@title ### **Remove RVC noise**
from pathlib import Path
import os
from glob import glob
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
noise_db_limit = -30 # @param {type:"slider", min:-50, max:0, step:0.1}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput"
Path(output_folder).mkdir(parents=True, exist_ok=True)

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py remove_noise \
            --audio_path "{input_file}" \
            --noise_db_limit {noise_db_limit} \
            --output_path "{output_folder}" \
            --supress True

In [ ]:
#@title ### **Ensemble**
from pathlib import Path
import os
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" #@param {type:"string"}
output_folder = "/content/drive/MyDrive/RVCAICoverMakerOutput" #@param {type:"string"}
Path(output_folder).mkdir(parents=True, exist_ok=True)
algorithm = "averege" # @param ['averege', 'Max Spec', 'Min Spec'] {allow-input: false}
files = [file for file in os.listdir(input_folder) if os.path.isfile(os.path.join(input_folder, file))]
common_name = os.path.commonprefix(files)

!python inference.py ensemble \
            --input_folder "{input_folder}" \
            --algorithm_ensemble "{algorithm}" \
            --output_path "{output_folder}/{common_name}.wav" \
            --supress True


In [ ]:
#@title ### **RVC inference**
from pathlib import Path
import os
from glob import glob
input_folder = "/content/drive/MyDrive/RVCAICoverMakerInput" # @param {type:"string"}
rvc_model_link = ""  # @param {type:"string"}
rvc_model_link = rvc_model_link.replace("https://", "")
rvc_model_name = os.path.splitext(os.path.basename(rvc_model_link))
f0method = "rmvpe"  # @param ["pm", "dio", "crepe", "crepe-tiny", "harvest", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
pitch = 0  # @param {type:"slider", min:-24, max:24, step:0}
filter_radius = 3  # @param {type:"slider", min:0, max:10, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
hop_length = 128  # @param {type:"slider", min:1, max:512, step:0}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
autotune = False  # @param{type:"boolean"}
output_drive = "/content/drive/MyDrive/RVCAICoverMakerOutput"
model_destination_folder = "/content/RVC_CLI/logs"

input_files = glob(os.path.join(input_folder, '*.flac')) + glob(os.path.join(input_folder, '*.mp3')) + glob(os.path.join(input_folder, '*.wav'))
for input_file in input_files:
  print("Processing", len(input_files), "files")
  !python inference.py rvc_ai \
            --input_path "{input_file}" \
            --output_path "{output_drive}" \
            --rvc_model_name "{rvc_model_name}" \
            --model_destination_folder "{model_destination_folder}" \
            --rvc_model_link "{rvc_model_link}" \
            --pitch {pitch} \
            --filter_radius {filter_radius} \
            --index_rate {index_rate} \
            --hop_length {hop_length} \
            --rms_mix_rate {rms_mix_rate} \
            --protect {protect} \
            --autotune {autotune} \
            --f0method "{f0method}" \
            --split_audio {split_audio} \
            --clean_audio {clean_audio} \
            --clean_strength {clean_strength} \
            --export_format "FLAC" \
            --supress True
